<h1><center><font size="6">Chinese MNIST EDA and Prediction</font></center></h1>

![](https://www.googleapis.com/download/storage/v1/b/kaggle-forum-message-attachments/o/inbox%2F769452%2Ffae77a81c057fe419de60f5e2b20be25%2Fchinese_mnist_profile_small.png?generation=1596963542354014&alt=media)
 

# <a id='0'>Content</a>

- <a href='#1'>Introduction</a>  
- <a href='#2'>Prepare the data analysis</a>  
 - <a href='#21'>Load packages</a>  
 - <a href='#21'>Load the data</a>  
 - <a href='#21'>Preprocessing data</a>  
- <a href='#3'>Data exploration</a>   
 - <a href='#31'>Check for missing data</a>  
 - <a href='#32'>Explore image data</a>  
  - <a href='#33'>Suites</a>  
- <a href='#4'>Characters classification</a>  
 - <a href='#40'>Split the data</a>  
 - <a href='#41'>Build a baseline model</a>  
 - <a href='#42'>Model evaluation</a>    
 - <a href='#43'>Add Dropout</a>  
 - <a href='#44'>Model refinement</a>  
- <a href='#6'>Conclusions</a>    
- <a href='#7'>References</a>    

# <a id='1'>Introduction</a>  


In this Kernel, we will explore a dataset with adnotated images of Chinese numbers, handwritten by a number of 100 volunteers, each providing a number of 10 samples, each sample with a complete set of 15 Chinese characters for numbers.

The Chinese characters are the following:
* 零 - for 0  
* 一 - for 1
* 二 - for 2  
* 三 - for 3  
* 四 - for 4  
* 五 - for 5  
* 六 - for 6  
* 七 - for 7  
* 八 - for 8  
* 九 - for 9  
* 十 - for 10
* 百 - for 100
* 千 - for 1000
* 万 - for 10 thousands
* 亿 - for 100 millions


The objective of the Kernel is to take us through the steps of a machine learning analysis.   
We start by preparing the analysis (load the libraries and the data), continue with an Exploratory Data Analysis (EDA) where we highlight the data features, spending some time to try to understand the data and also get an idea about various features predictive potential and correlation with other features.   

We follow then with features engineering and preparation for creation of a model. The dataset is split in training, validation and test set. We start then with a simple model to classify the Chinese numbers images, something we are calling a baseline model.   

We evaluate the model, estimating the training error and accuracy and also the validation error and accuracy. With these, and with an rough estimate of what will be the (human) error rate for classification of Chinese numbers images, we decide how to follow our machine learning for image classification work. If we have at start a high bias, we will try first to improve our model so that will learn better the train images dataset. If we have a small bias but large variance (the model learns well the training data but fails to generalize, that means our model is overfitting. Based on these kind of observation, we make decission for how to adjust the model.   

We run few models with the improvements decided based on analysis or error and accuracy and we decide at the end for a final model. This model will be used for classification of fresh, new data, not used for training or validation, the test set.

<a href="#0"><font size="1">Go to top</font></a>  

# <a id='2'>Prepare the data analysis</a>   


Before starting the analysis, we need to make few preparation: load the packages, load and inspect the data.



# <a id='21'>Load packages</a>

We load the packages used for the analysis.


In [ ]:
import pandas as pd
import numpy as np
import sys
import os
import random
from pathlib import Path
import imageio
import skimage
import skimage.io
import skimage.transform
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import scipy
from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPool2D, Dropout, BatchNormalization,LeakyReLU
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from keras.utils import to_categorical
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import tensorflow

In [ ]:
IMAGE_PATH = '..//input//chinese-mnist//data//data//'
IMAGE_WIDTH = 64
IMAGE_HEIGHT = 64
IMAGE_CHANNELS = 1
RANDOM_STATE = 42
TEST_SIZE = 0.2
VAL_SIZE = 0.2
CONV_2D_DIM_1 = 16
CONV_2D_DIM_2 = 16
CONV_2D_DIM_3 = 32
CONV_2D_DIM_4 = 64
MAX_POOL_DIM = 2
KERNEL_SIZE = 3
BATCH_SIZE = 32
NO_EPOCHS_1 = 5
NO_EPOCHS_2 = 10
NO_EPOCHS_3 = 50
PATIENCE = 5
VERBOSE = 1

<a href="#0"><font size="1">Go to top</font></a>  


# <a id='22'>Load the data</a>  

Let's see first what data files do we have in the root directory.

In [ ]:
os.listdir("..//input//chinese-mnist")

There is a dataset file and a folder with images.  

Let's load the dataset file first.

In [ ]:
data_df=pd.read_csv('..//input//chinese-mnist//chinese_mnist.csv')

Let's glimpse the data. First, let's check the number of columns and rows.

In [ ]:
data_df.shape

There are 15000 rows and 5 columns. Let's look to the data.

In [ ]:
data_df.sample(100).head()

The data contains the following values:  

* suite_id - each suite corresponds to a set of handwritten samples by one volunteer;  
* sample_id - each sample wil contain a complete set of 15 characters for Chinese numbers;
* code - for each Chinese character we are using a code, with values from 1 to 15;
* value - this is the actual numerical value associated with the Chinese character for number;  
* character - the Chinese character;  

We index the files in the dataset by forming a file name from suite_id, sample_id and code. The pattern for a file is as following:

> "input_{suite_id}_{sample_id}_{code}.jpg"

<a href="#0"><font size="1">Go to top</font></a>  

# <a id='3'>Data exploration</a>  



Let's start by checking if there are missing data, unlabeled data or data that is inconsistently labeled. 


## <a id='31'>Check for missing data</a>  

Let's create a function that check for missing data in the dataset.

In [ ]:
def missing_data(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    return pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data(data_df)

There is no missing (null) data in the dataset. Still it might be that some of the data labels are misspelled; we will check this when we will analyze each data feature.

<a href="#0"><font size="1">Go to top</font></a>  

## <a id='32'>Explore image data</a>  

Let's also check the image data. First, we check how many images are stored in the image folder.

In [ ]:
image_files = list(os.listdir(IMAGE_PATH))
print("Number of image files: {}".format(len(image_files)))

Let's also check that each line in the dataset has a corresponding image in the image list.  
First, we will have to compose the name of the file from the indexes.

In [ ]:
def create_file_name(x):
    
    file_name = f"input_{x[0]}_{x[1]}_{x[2]}.jpg"
    return file_name

In [ ]:
data_df["file"] = data_df.apply(create_file_name, axis=1)

In [ ]:
file_names = list(data_df['file'])
print("Matching image names: {}".format(len(set(file_names).intersection(image_files))))

Let's also check the image sizes.

In [ ]:
def read_image_sizes(file_name):
    image = skimage.io.imread(IMAGE_PATH + file_name)
    return list(image.shape)

In [ ]:
m = np.stack(data_df['file'].apply(read_image_sizes))
df = pd.DataFrame(m,columns=['w','h'])
data_df = pd.concat([data_df,df],axis=1, sort=False)

In [ ]:
data_df.head()

## <a id='33'>Suites</a>  

Let's check the suites of the images. For this, we will group by `suite`.

In [ ]:
print(f"Number of suites: {data_df.suite_id.nunique()}")
print(f"Samples: {data_df.sample_id.unique()}")

We have 100 suites, each with 10 samples.

# <a id='4'>Characters classification</a>

Our objective is to use the images that we investigated until now to correctly identify the Chinese numbers (characters).   

We have a unique dataset and we will have to split this dataset in **train** and **test**. The **train** set will be used for training a model and the test will be used for testing the model accuracy against new, fresh data, not used in training.



## <a id='40'>Split the data</a>  

First, we split the whole dataset in train and test. We will use **random_state** to ensure reproductibility of results.   

The train-test split is **80%** for training set and **20%** for test set.


In [ ]:
train_df, test_df = train_test_split(data_df, test_size=TEST_SIZE, random_state=RANDOM_STATE)

Next, we will split further the **train** set in **train** and **validation**. We want to use as well a validation set to be able to measure not only how well fits the model the train data during training (or how well `learns` the training data) but also how well the model is able to generalize so that we are able to understands not only the bias but also the variance of the model.  

The train-validation split is **80%** for training set and **20%** for validation set.

In [ ]:
train_df, val_df = train_test_split(train_df, test_size=VAL_SIZE, random_state=RANDOM_STATE)

Let's check the shape of the three datasets.

In [ ]:
print("Train set rows: {}".format(train_df.shape[0]))
print("Test  set rows: {}".format(test_df.shape[0]))
print("Val   set rows: {}".format(val_df.shape[0]))

In [ ]:
def plot_count(feature, title, df, size=1):
    f, ax = plt.subplots(1,1, figsize=(4*size,4))
    total = float(len(df))
    g = sns.countplot(df[feature], order = df[feature].value_counts().index[:20], palette='Set3')
    g.set_title("Number and percentage of {}".format(title))
    if(size > 2):
        plt.xticks(rotation=90, size=8)
    for p in ax.patches:
        height = p.get_height()
        ax.text(p.get_x()+p.get_width()/2.,
                height + 3,
                '{:1.2f}%'.format(100*height/total),
                ha="center") 
    plt.show()    

In [ ]:
plot_count("value", "value (train data)", train_df, size=3)

In [ ]:
plot_count("value", "value (validation data)", val_df, size=3)

In [ ]:
plot_count("value", "value (test data)", test_df, size=3)

**Note**:  Training, validation and test data are slightly unbalanced. We can improve on this part, if we use a different train-test split approach, imposing to have balanced distribution in the splits, based on the label feature. Let's do it now.

In [ ]:
train_df, test_df = train_test_split(data_df, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=data_df["code"].values)
train_df, val_df = train_test_split(train_df, test_size=VAL_SIZE, random_state=RANDOM_STATE, stratify=train_df["code"].values)
print("Train set rows: {}".format(train_df.shape[0]))
print("Test  set rows: {}".format(test_df.shape[0]))
print("Val   set rows: {}".format(val_df.shape[0]))

In [ ]:
plot_count("value", "value (train data)", train_df, size=3)

In [ ]:
plot_count("value", "value (validation data)", val_df, size=3)

In [ ]:
plot_count("value", "value (test data)", test_df, size=3)

Now, using `stratify` option, we also balanced train, validation and test set with respect with the classes distribution.  
We are now ready to start building our first model.

## <a id='41'>Build a baseline model</a>    


Next step in our creation of a predictive model is to create a simple model, a **baseline model**.  

 Why start with a simple model (as simple as possible, but not simpler :-) )?
 
 With a simple model, we can get fast insight in how well will the data predict our target value. Looking to the training results (the training error and accuracy, the validation error and accuracy), we can understand if we need to add more data (because the training accuracy is small) or if we need to optimize the model (by adding more convolutional layers) or if we need to add Dropout layers (because the validation error is increasing after few steps - the model is overfitting) etc.
 
Let's define few auxiliary functions that we will need for creation of our models.

In [ ]:
def read_image(file_name):
    image = skimage.io.imread(IMAGE_PATH + file_name)
    image = skimage.transform.resize(image, (IMAGE_WIDTH, IMAGE_HEIGHT, 1), mode='reflect')
    return image[:,:,:]

A function to create the dummy variables corresponding to the categorical target variable.

In [ ]:
def categories_encoder(dataset, var='character'):
    X = np.stack(dataset['file'].apply(read_image))
    y = pd.get_dummies(dataset[var], drop_first=False)
    return X, y

Let's populate now the train, val and test sets with the image data and create the  dummy variables corresponding to the categorical target variable, in our case `subspecies`.

In [ ]:
X_train, y_train = categories_encoder(train_df)
X_val, y_val = categories_encoder(val_df)
X_test, y_test = categories_encoder(test_df)

Now we are ready to start creating our model.  

We will add the folllowing elements to our model: 
* One convolutional layer, with 16 filters of dimmension 3;  
* One maxpoll2d layer, with reduction factor 2;  
* One convolutional layer, with 16 filters of dimmension 3;  
* A flatten layer;  
* A dense layer;  

In [ ]:
model1=Sequential()
model1.add(Conv2D(CONV_2D_DIM_1, kernel_size=KERNEL_SIZE, input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 1), activation='relu', padding='same'))
model1.add(MaxPool2D(MAX_POOL_DIM))
model1.add(Conv2D(CONV_2D_DIM_2, kernel_size=KERNEL_SIZE, activation='relu', padding='same'))
model1.add(Flatten())
model1.add(Dense(y_train.columns.size, activation='softmax'))
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model1.summary()

We train the first model using and a predefined batch size. We are using the predefined epoch number for this first experiment (5 steps) and as well validation, using the validation set. 

In [ ]:
train_model1 = model1.fit(X_train, y_train,
                  batch_size=BATCH_SIZE,
                  epochs=NO_EPOCHS_1,
                  verbose=1,
                  validation_data=(X_val, y_val))

<a href="#0"><font size="1">Go to top</font></a>  


## <a id='42'>Model evaluation</a> 


Let's start by plotting the loss error for the train and validation set. 
We define a function to visualize these values.

In [ ]:
def create_trace(x,y,ylabel,color):
        trace = go.Scatter(
            x = x,y = y,
            name=ylabel,
            marker=dict(color=color),
            mode = "markers+lines",
            text=x
        )
        return trace
    
def plot_accuracy_and_loss(train_model):
    hist = train_model.history
    acc = hist['accuracy']
    val_acc = hist['val_accuracy']
    loss = hist['loss']
    val_loss = hist['val_loss']
    epochs = list(range(1,len(acc)+1))
    #define the traces
    trace_ta = create_trace(epochs,acc,"Training accuracy", "Green")
    trace_va = create_trace(epochs,val_acc,"Validation accuracy", "Red")
    trace_tl = create_trace(epochs,loss,"Training loss", "Blue")
    trace_vl = create_trace(epochs,val_loss,"Validation loss", "Magenta")
    fig = tools.make_subplots(rows=1,cols=2, subplot_titles=('Training and validation accuracy',
                                                             'Training and validation loss'))
    #add traces to the figure
    fig.append_trace(trace_ta,1,1)
    fig.append_trace(trace_va,1,1)
    fig.append_trace(trace_tl,1,2)
    fig.append_trace(trace_vl,1,2)
    #set the layout for the figure
    fig['layout']['xaxis'].update(title = 'Epoch')
    fig['layout']['xaxis2'].update(title = 'Epoch')
    fig['layout']['yaxis'].update(title = 'Accuracy', range=[0,1])
    fig['layout']['yaxis2'].update(title = 'Loss', range=[0,1])
    #plot
    iplot(fig, filename='accuracy-loss')

plot_accuracy_and_loss(train_model1)


Let's continue by evaluating the **test** set **loss** and **accuracy**. We will use here the test set.

In [ ]:
score = model1.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Let's check also the test accuracy per class.

In [ ]:
def test_accuracy_report(model):
    predicted = model.predict(X_test)
    test_predicted = np.argmax(predicted, axis=1)
    test_truth = np.argmax(y_test.values, axis=1)
    print(metrics.classification_report(test_truth, test_predicted, target_names=y_test.columns)) 
    test_res = model.evaluate(X_test, y_test.values, verbose=0)
    print('Loss function: %s, accuracy:' % test_res[0], test_res[1])

In [ ]:
test_accuracy_report(model1)

We used a simple model. We separated 20% of the data for testing. From the training data, 80% is used for actual training and 20% for testing.   

Adding additional data will only slightly increase the accuracy of the training set (it is already very good).   
To reduce the loss of the validation set (which is a sign of overfitting), we can have three strategies:  
* add Dropout layers;  
* introduce strides;  
* modify the learning rate during the training;  


<a href="#0"><font size="1">Go to top</font></a>  


## <a id='43'>Add Dropout</a>  

We add two Dropout layers.  The role of the Dropout layers is to reduce the overfitting, by dropping, each training epoch, a certain percent of the nodes connections (by rotation). This is equivalent of using less training data and in the same time training the network with various data as well as using `parallel` alternative networks, thus reducing the likelihood that the network will overfit the train data.  

The definition of the second model is:

In [ ]:
model2=Sequential()
model2.add(Conv2D(CONV_2D_DIM_1, kernel_size=KERNEL_SIZE, input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT,IMAGE_CHANNELS), activation='relu', padding='same'))
model2.add(MaxPool2D(MAX_POOL_DIM))
# Add dropouts to the model
model2.add(Dropout(0.4))
model2.add(Conv2D(CONV_2D_DIM_2, kernel_size=KERNEL_SIZE, activation='relu', padding='same'))
# Add dropouts to the model
model2.add(Dropout(0.4))
model2.add(Flatten())
model2.add(Dense(y_train.columns.size, activation='softmax'))
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Let's inspect the new model.

In [ ]:
model2.summary()

We can observe that this model has the same number of parameters and trainable parameters as  the previous model.

In [ ]:
train_model2  = model2.fit(X_train, y_train,
                  batch_size=BATCH_SIZE,
                  epochs=NO_EPOCHS_2,
                  verbose=1,
                  validation_data=(X_val, y_val))

### Evaluate model accuracy and loss

In [ ]:
plot_accuracy_and_loss(train_model2)

### Test accuracy and loss

Let's evaluare as well the test accuracy and loss.

In [ ]:
test_accuracy_report(model2)

<a href="#0"><font size="1">Go to top</font></a>  

## <a id='45'>Model refinement</a>  


We define now also a refined model. 

We add an early stopping condition (monitor the loss error and stops the training if for a number of stept given in the `patience` parameters the loss is not improving).

We are also saving a model checkpoint after each epoch when accuracy improves; if accuracy degrades, no new model is saved. Thus, Model Checkpoint saves all the time the best model in terms of accuracy.  

We adjust as well the learning rate with the training epochs.

Also, we increase the number of training epochs to 50.



In [ ]:
annealer3 = LearningRateScheduler(lambda x: 1e-3 * 0.995 ** (x+NO_EPOCHS_3))
earlystopper3 = EarlyStopping(monitor='loss', patience=PATIENCE, verbose=VERBOSE)
checkpointer3 = ModelCheckpoint('best_model_3.h5',
                                monitor='val_acc',
                                verbose=VERBOSE,
                                save_best_only=True,
                                save_weights_only=True)

In [ ]:
model3=Sequential()
model3.add(Conv2D(CONV_2D_DIM_1, kernel_size=KERNEL_SIZE, input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT,IMAGE_CHANNELS), activation='relu', padding='same'))
model3.add(MaxPool2D(MAX_POOL_DIM))
# Add dropouts to the model
model3.add(Dropout(0.4))
model3.add(Conv2D(CONV_2D_DIM_2, kernel_size=KERNEL_SIZE, activation='relu', padding='same'))
# Add dropouts to the model
model3.add(Dropout(0.4))
model3.add(Flatten())
model3.add(Dense(y_train.columns.size, activation='softmax'))
model3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Let's inspect the refined model.

In [ ]:
model3.summary()

Now, let's train the model.

In [ ]:
train_model3  = model3.fit(X_train, y_train,
                  batch_size=BATCH_SIZE,
                  epochs=NO_EPOCHS_3,
                  verbose=1,
                  validation_data=(X_val, y_val),
                  callbacks=[earlystopper3, checkpointer3, annealer3])

### Model accuracy and loss

In [ ]:
plot_accuracy_and_loss(train_model3)

### Test accuracy and loss

In [ ]:
test_accuracy_report(model3)

<a href="#0"><font size="1">Go to top</font></a>  

# <a id='6'>Conclusions</a>  

After exploring the data to understand its various features, a baseline model is created.  We start with a baseline model because we want to evaluate first how a simple model performs, what is the precision/accuracy for training, validation and test set. 

Evaluation of the baseline model  results for valid set and test set allows us to decide, based on analysis of bias and variance, how to conduct furher our experiments. After the analysis of the baseline solution, we decided that, while the training is good enough, we would like to improve on the variance, thus reducing overfitting.

From the possible solutions for overfitting, we choose to add Dropout layers. Adding Dropout layers improve a bit the algorithm performance (reduce overfitting).  

A third model, with adjustable learning rate, early stoping based on validation accuracy measurement. The model is saved every time when validation accuracy improves. Also, the training will stop if after a number of steps the validation accuracy is not improving. With this model, accuracy of prediction for the test set was improved. But looking to the validation loss, we can see we do have a problem - we most probably we still overfit on train data, so we will have to further improve this model.

The **key lessons learned** from this Kernel are the following:   
* start by analyzing the data;   
* follow with a simple baseline model;   
* refine gradually the model, by making corrections based on the analysis of the (partial) results.

**Note**: we didn't used an accelerator for this Kernel. Therefore, if you want to improve the calculation speed, select to use one accelerator (**GPU**/**TPU**) for your Kernel.

<a href="#0"><font size="1">Go to top</font></a>

# <a id='7'>References</a>  

[1] Gabriel Preda, RSNA Pneumonia Detection EDA, https://www.kaggle.com/gpreda/rsna-pneumonia-detection-eda     
[2] Gabriel Preda, CNN with Tensorflow|Keras for Fashion-MNIST, https://www.kaggle.com/gpreda/cnn-with-tensorflow-keras-for-fashion-mnist    
[3] DanB, CollinMoris, Deep Learning From Scratch, https://www.kaggle.com/dansbecker/deep-learning-from-scratch  
[4] DanB, Dropout and Strides for Larger Models, https://www.kaggle.com/dansbecker/dropout-and-strides-for-larger-models  
[5] BGO, CNN with Keras, https://www.kaggle.com/bugraokcu/cnn-with-keras  
[6] Dmitri Pukhov, Honey Bee health detection using CNN, https://www.kaggle.com/gpreda/honey-bee-health-detection-with-cnn/notebook     
[7] Why Dropounts prevent overfitting in Deep Neural Networks, https://medium.com/@vivek.yadav/why-dropouts-prevent-overfitting-in-deep-neural-networks-937e2543a701  
[8] Dropout: A Simple Way to Prevent Neural Networks from Overfitting, https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf  
[9] Gabriel Preda, Honey Bee Subspecies Classification, https://www.kaggle.com/gpreda/honey-bee-subspecies-classification  
<a href="#0"><font size="1">Go to top</font></a>
